In [1]:
import services as svs

/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/floodProbProj/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['./whitebox_tools', '-v']


In [ ]:
# Import tails form an xxcell doc. ]
excell_file = '/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/saint_john_NFL_DTM.xlsx'
listTails = svs.importListFromExelCol(excell_file,'Feuil1','ftp_dtm')
importer = svs.dtmTailImporter(listTails)
importer.impotTailToLocalDir()

In [2]:
## my work directory: /Users/abdielfer/DESS/Internship2022/RNCanWork/FloodMaps/testZone
transformer = svs.dtmTransformer()

In [ ]:
outpouDTMName = 'zone_one.tif' 
transformer.computeMosaic(outpouDTMName)
# Output from Mosaic is input for the rest of the process
transformer.fixNoDataAndfillDTM(outpouDTMName)

In [ ]:
inDTMFilled = "filled_"+outpouDTMName
transformer.d8FPointerRasterCalculation(inDTMFilled)

In [5]:
inD8Pointer = "d8Pointer_filled_zone_one.tif" 
transformer.basinsDelineation (inD8Pointer)

./whitebox_tools --run="Basins" --wd="/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodMaps/testZone" --d8_pntr='d8Pointer_filled_zone_one.tif' --output='basins_d8Pointer_filled_zone_one.tif' --compress_rasters

****************************
* Welcome to Basins        *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Initializing: 0%
Initializing: 1%
Initializing: 2%
Initializing: 3%
Initializing: 4%
Initializing: 5%
Initializing: 6%
Initializing: 7%
Initializing: 8%
Initializing: 9%
Initializing: 10%
Initializing: 11%
Initializing: 12%
Initializing: 13%
Initializing: 14%
Initializing: 15%
Initializing: 16%
Initializing: 17%
Initializing: 18%
Initializing: 19%
Initializing: 20%
Initializing: 21%
Initializing: 22%
Initializing: 23%
Initializing: 24%
Initializing: 25%
Initializing: 26%
Initializing: 27%
Initializing: 28%
Initializing: 29%
Initializing: 30%
Initializing: 31%
Initializing: 32%
Initializing: 33%
Initializing: 34%
Init

In [ ]:

# transformer.computeSlope(dtmName)
# transformer.computeAspect(dtmName)